# Data Preparation

The aim of this project phase is to load and transform the project data, making it ready for further analysis and main project execution.

## The Dataset

The dataset in use was obtained from [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/dataset/502/online+retail+ii). It contains the records of all transactions done for a UK-based, giftware online retail store.

## Load Dependencies

In [1]:
# Module containing all dependencies
import src.dependencies as dep

# Module containing custom functions
import src.functions as fn

## Load the Data

In [2]:
# Get the data file
data_file = 'dataset/online_retail_II.xlsx'

# Load data in respective sheets
sheet1 = dep.pd.read_excel(data_file, sheet_name = 'Year 2009-2010')
sheet2 = dep.pd.read_excel(data_file, sheet_name = 'Year 2010-2011')

# Join data in both sheets
df = dep.pd.concat([sheet1, sheet2])

# Confirm data has been loaded successfully
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## Data Cleaning

### Duplicates

Check for and eliminate duplicate rows. In instances where there are duplicate rows, the first duplicate will be kept.

In [3]:
# Eliminate duplicate rows
before, after, df = fn.duplicate_rows(df)
print('There were {} duplicate rows, and {} duplicate rows after elimination.'.format(before, after))

There were 34335 duplicate rows, and 0 duplicate rows after elimination.


### Missing Values

Check if there are any missing values in the data.

In [4]:
# Count of missing values
df.isnull().sum()

Invoice             0
StockCode           0
Description      4275
Quantity            0
InvoiceDate         0
Price               0
Customer ID    235151
Country             0
dtype: int64

There are some entries without a description, but all have a stock code, therefore the lack of the description is not crucial.

Conversely, a customer identifier is essential in predicting the lifetime value. As a result, all entries without a customer identifier will be dropped.

In [5]:
# Drop entries without customer ID
df = df.dropna(subset = ['Customer ID'])

# Confirm all entries have customer ID
df['Customer ID'].isnull().sum()

0

### Data Types

Confirm that the data has been represented in the correct data types.

In [6]:
# Check respective data types
df.dtypes

Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object

Referencing the sample data displayed in the **Loading the Data** section, all columns have the correct data types except `Invoice` and `Customer ID`.

`Invoice` data type will be changed from _object_ to _integer_.

`Customer ID` data type will be changed from _float_ to _integer_.

In [7]:
# Ensure entries in Invoice column are numeric
df['Invoice'] = df['Invoice'].astype(str)
df['Invoice'] = df['Invoice'].str.extract('(\d+)', expand = False)

# Convert respective columns
df['Invoice'] = df['Invoice'].astype(int)
df['Customer ID'] = df['Customer ID'].astype(int)

# Confirm conversion
df.dtypes

Invoice                 int32
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID             int32
Country                object
dtype: object

### Outliers

Using the data description, check if there are any outliers in the data, specifically, quantity and price entries.

In [8]:
# Statistical summary
df[['Quantity', 'Price']].describe()

,Quantity,Price
count,797885.000000,797885.000000
mean,12.602980,3.702732
std,191.670371,71.392549
min,-80995.000000,0.000000
25%,2.000000,1.250000
50%,5.000000,1.950000
75%,12.000000,3.750000
max,80995.000000,38970.000000


The expectation is that both `Quantity` and `Price` only contain positive entries. From the result set above, `Quantity` has some negative entries. 

Upon consultation, it's been established that the negative entries in the `Quantity` column imply a sales return, therefore there are no outliers in the data.

## Transformed Data

The cleaned data will be saved in a csv file.

In [9]:
# Save the dataframe to a CSV file
df.to_csv('dataset/Transformed.csv', index = False)